In [1]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import json
import os
from utils import get_assistant_texts, get_tag_masks, remove_tags, replace_tags, get_tags, get_message_ids, get_val_scores, get_prev_messages
import time
from tqdm import tqdm
import threading
import queue
from itertools import islice

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make sharegpt dataset

In [2]:
import json
import numpy as np
import pandas as pd

seed = 633
np.random.seed(seed)

sharegpt = pd.read_json("data/generated-responses/rank_0_0_transcripts_sharegpt_meta-llama_Llama-2-7b-chat-hf.json")

sharegpt[:1]

,message_id,prompt,assistant_text,parent_id,prev_messages,role,model,template_name,gen_kwargs,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject
0,dc71dd66-8eab-464b-9023-a03e820e99ff,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",The length of time before considering a situat...,nxqSeIG_0,[So i had this friend named Jess in semester o...,assistant,meta-llama/Llama-2-7b-chat-hf,null,max_length=4096;temperature=1.0;repetition_pen...,null,null,null,null,null


In [14]:
from AptTagger import AptTagger

apt_tagger = AptTagger(model_name=None)
strout = ""
for row in sharegpt.iloc[:200].iloc:
    this = ""
    this += f"[[ID]] {row['parent_id']}\n"
    this += f"[[TRANSCRIPT]] "
    for i, message in enumerate(row['prev_messages']):
        role = "human" if i % 2 == 0 else "gpt"
        if role == "human":
            this += "USER: "            
        elif role == "gpt":
            this += "ASSISTANT: "
        else:
            raise Exception("Unknown role", role)
        
        this += f"{message}\n\n"
    
    message = row["assistant_text"]
    tag_mask = apt_tagger.tag(message)
    annotated = AptTagger.annotate(message, tag_mask)
    this += f"ASSISTANT: {annotated}"
    this += "[[NOT YET ANNOTATED]]\n\n\n"
    strout += this

print(strout)

[[ID]] nxqSeIG_0
[[TRANSCRIPT]] USER: So i had this friend named Jess in semester one . We were on the same dorm floor. We use to hang out have fun and banter . I use to joke around with her. She got really mad at a joke once and then i weirded out cause it wasnt something to get that mad about, but i found out that she found out that i had a lil crush on her at that time. She stopped banter and most communication. Then i started saying sorry , communicating extra and slowly things just touched being normal. On her birthday I got her a cake and a gift and organized kind of a party type. She didnt invite me out for her bbirthday but she was really happy. Following that week we had a month off and on the first day I made a joke about coming to her hometown because I'm missing her just the day after she left, just to freak her out and have fun. But she got really mad after I told her it was a joke. I showed her fake airplane tickets and actually made her believe. She thought I have a crus

In [13]:
with open("data/natural/sharegpt.txt", "w") as f:
    f.write(strout)

# Binarize MMLU
3 ways, from least to most general:

"{mcq}\n\nThe answer is {correct/incorrect_answer with punc removed}"

can't just remove extra options because sometimes they're needed contextually. I should probably try to filter these out based on "least" "all of the above" "none of the above" "most" "best"

same as above but including USER: Q and ASSISTANT: QA

In [83]:
from datasets import load_dataset, DatasetDict
ds_name = "cais/mmlu"
ds_dict = load_dataset(ds_name, "all")
ds_dict = DatasetDict({split: ds_dict[split] for split in ["validation", "dev", "test"]})  # throw out auxiliary train
# TODO: try using auxiliary train too
ds_dict["validation"][:2]

{'question': ['Which of the following situations does NOT occur in a federal state?',
  'How do city planners separate discordant land uses such as housing and sewage plants?'],
 'subject': ['high_school_geography', 'high_school_geography'],
 'choices': [['Central government possesses a two-level system of government.',
   'Central government governs country as a single unit.',
   'It often possesses a written constitution.',
   'Lower-level divisions have unique powers.'],
  ['Malls', 'Fences', 'Zoning', 'Tunnels']],
 'answer': [1, 2]}

In [82]:
ds_dict["validation"][:2]

{'question': ['Which of the following situations does NOT occur in a federal state?',
  'How do city planners separate discordant land uses such as housing and sewage plants?'],
 'subject': ['high_school_geography', 'high_school_geography'],
 'choices': [['Central government possesses a two-level system of government.',
   'Central government governs country as a single unit.',
   'It often possesses a written constitution.',
   'Lower-level divisions have unique powers.'],
  ['Malls', 'Fences', 'Zoning', 'Tunnels']],
 'answer': [1, 2]}

In [117]:
# method 2
import random
import re
from datasets import ClassLabel, Value

def filter(ex):
    question_exclude = "least, most, est, following, below, __, these is not".split(", ")
    answer_exclude = "none, all".split(", ")
    for q in question_exclude:
        if q in ex["question"].lower():
            return False
    for a in answer_exclude:
        for choice in ex["choices"]:
            if a in choice.lower():
                return False
    if ex["subject"] == "business_ethics":
        for choice in ex["choices"]:
            if re.match(r"^\d,\d", choice):
                return False
    return True

def binarize1(exs, make_transcript=False):
    results = []
    for i in range(len(exs["question"])):
        q = exs["question"][i]
        choices = exs['choices'][i]
        ans = exs['answer'][i]
        true_choice = choices[ans].removesuffix(".")
        false_choice = choices[random.choice([i for i in range(len(choices)) if i != ans])].removesuffix(".")
        template = "USER: {q}\n\nASSISTANT: {q} {a}" if make_transcript else "{q} {a}"
        true_st = template.format(q=q, a=true_choice)
        false_st = template.format(q=q, a=false_choice)
        ex = {k: exs[k][i] for k in exs.keys()}
        true_ex = {**ex, "statement": true_st, "label": 1}
        false_ex = {**ex, "statement": false_st, "label": 0}
        results.append(true_ex)
        results.append(false_ex)
    # transpose results
    results = {k: [ex[k] for ex in results] for k in results[0].keys()}
    return results

ds_dict1 = ds_dict.filter(filter)
ds_dict1 = ds_dict1.map(binarize1, batched=True).shuffle()
ds_dict1 = ds_dict1.cast_column("label", ClassLabel(names=["false", "true"]))
ds_dict1 = ds_dict1.cast_column("answer", Value("int32"))
ds_dict1.push_to_hub("atmallen/mmlu_binary")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  3.38it/s]


In [120]:
ds_dict2 = ds_dict.filter(filter)
ds_dict2 = ds_dict2.map(binarize1, batched=True, fn_kwargs={"make_transcript": True}).shuffle()
ds_dict2 = ds_dict2.cast_column("label", ClassLabel(names=["false", "true"]))
ds_dict2 = ds_dict2.cast_column("answer", Value("int32"))
ds_dict2.push_to_hub("atmallen/mmlu_chat_binary")

Casting the dataset:   0%|          | 0/11526 [00:00<?, ? examples/s]

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  6.80it/s]


In [99]:
# now use the various prompt templates I used


{'question': ['Ethics of duty is a principle based ethical theory attributed to Emmanuel Kant.',
  'Ethics of duty is a principle based ethical theory attributed to Emmanuel Kant.',
  'Executive directors are responsible for running the firm.',
  'Executive directors are responsible for running the firm.',
  'Subjection to emotional exploitation.',
  'Subjection to emotional exploitation.'],
 'subject': ['business_ethics',
  'business_ethics',
  'business_ethics',
  'business_ethics',
  'business_ethics',
  'business_ethics'],
 'choices': [['1,3', '2,3', '1,2', '1,2,3'],
  ['1,3', '2,3', '1,2', '1,2,3'],
  ['1,4', '1,3', '2,3', '1,2,3'],
  ['1,4', '1,3', '2,3', '1,2,3'],
  ['1,2,4', '1,3,4', '2,3,4', '1,2,3,4,5'],
  ['1,2,4', '1,3,4', '2,3,4', '1,2,3,4,5']],
 'answer': [0, 0, 0, 0, 1, 1],
 'text': ['Ethics of duty is a principle based ethical theory attributed to Emmanuel Kant. 1,3',
  'Ethics of duty is a principle based ethical theory attributed to Emmanuel Kant. 1,2,3',
  'Executive

# Make natural validations set

In [5]:
import re
user_prefix = "USER:"

def get_statements(text, strip=True, include_prev_messages=False):
    """returns individual statements without context"""
    ast_texts = get_assistant_texts(text)
    prev_messages = get_prev_messages(text)
    pattern = "\[\[\d+\]\]"
    statements = []
    for prev, t in zip(prev_messages, ast_texts):
        sts = re.split(pattern, t)[:-1]
        if include_prev_messages:
            sts[0] = prev + "\n\nASSISTANT: " + sts[0]
        if strip:
            sts = [s.strip() for s in sts]
        statements.append(sts)
    return statements

def get_full_statements(text, include_prev_messages=True):
    """returns a list of lists of fully contextualized statements in each transcript"""
    statements = get_statements(text, strip=False, include_prev_messages=include_prev_messages)
    prefs = [["".join(sts[:i]) for i in range(1, len(sts) + 1)] for sts in statements]
    return prefs

In [13]:
pathname = "data/natural/oasst_alpaca_labeled.txt"
with open(pathname, "r") as f:
    text = f.read()
val_df = get_val_scores(pathname)
statements = get_full_statements(text)
assert all([len(sts) == len(v) for sts, v in zip(statements, val_df.scores.values)])
val_df["statements"] = statements
val_df

,scores,statements
d48005ef42404f53fd1e16814f066e83,"[Certainly True, Certainly False, False, Certa...",[USER: Who's Marshall Rosenberg and what is hi...
51143b0c-e9eb-445e-a27b-2b85264cb165,"[False, Uncertain, N/A, N/A, N/A]",[USER: You are required to analyze the followi...
8a80c4cb-424c-40ac-bb23-7d473b3b2691,[N/A],[USER: Create a funny joke with a pun.\n\nASSI...
4cbaf274-bfca-44e0-8367-8f4ecbf408a1,[True],[USER: Find the geographical coordinates for t...
a88a4fc2-2880-4378-a18d-fb76c8cc98f0,"[False, False, Certainly True]",[USER: Name an animal that is considered a nui...
...,...,...
6bf2cc3b-c2a9-4915-8eee-fbf0fe58eeb1,"[N/A, True, Certainly False, True, Certainly F...","[USER: As of 2019, about what percentage of Am..."
6693b4a0-bef9-41c6-8ea5-5c3c3716d50b,"[N/A, True, N/A, N/A, False, True, N/A, N/A, N...",[USER: For which of these two scenarios does t...
350a2123-93d9-4994-a9a9-756adad86570,"[Certainly True, Certainly True, Certainly Tru...",[USER: It is difficult to ascertain the true e...
96c6fe02-c3f7-41b1-9e43-d52b717c2d9e,"[N/A, True, Certainly False, False, Certainly ...",[USER: Which of the following is NOT a spread ...


In [25]:
import numpy as np
scores = np.concatenate(val_df.scores.values)
statements = np.concatenate(val_df.statements.values)
mids = np.concatenate([[mid] * len(sts) for mid, sts in zip(val_df.index.values, val_df.statements.values)])
statement_index = np.concatenate([np.arange(len(sts)) for sts in val_df.statements.values])
assert len(scores) == len(statements) == len(mids) == len(statement_index)
df = pd.DataFrame({"message_id": mids, "s_idx": statement_index, "statement": statements, "score": scores})
df = df[df["score"].isin({"Certainly True", "Certainly False"})]
df["label"] = df["score"] == "Certainly True"
print(df["label"].value_counts())
df

label
True     145
False     52
Name: count, dtype: int64


,message_id,s_idx,statement,score,label
0,d48005ef42404f53fd1e16814f066e83,0,USER: Who's Marshall Rosenberg and what is his...,Certainly True,True
1,d48005ef42404f53fd1e16814f066e83,1,USER: Who's Marshall Rosenberg and what is his...,Certainly False,False
3,d48005ef42404f53fd1e16814f066e83,3,USER: Who's Marshall Rosenberg and what is his...,Certainly False,False
15,a88a4fc2-2880-4378-a18d-fb76c8cc98f0,2,USER: Name an animal that is considered a nuis...,Certainly True,True
25,9dc817cf-8457-4d25-98a5-febfc8c8b315,2,USER: Explain the concept of hyperparameter op...,Certainly False,False
...,...,...,...,...,...
452,96c6fe02-c3f7-41b1-9e43-d52b717c2d9e,4,USER: Which of the following is NOT a spread e...,Certainly False,False
453,96c6fe02-c3f7-41b1-9e43-d52b717c2d9e,5,USER: Which of the following is NOT a spread e...,Certainly False,False
459,f15191ab-d08f-4874-b38a-8b3bbad08728,2,USER: Individuals in which of the following oc...,Certainly False,False
460,f15191ab-d08f-4874-b38a-8b3bbad08728,3,USER: Individuals in which of the following oc...,Certainly True,True


In [21]:
df.to_json("data/natural/oasst_alpaca_labeled.jsonl", orient="records", lines=True)

In [53]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value

ds = Dataset.from_pandas(df)
ds = ds.remove_columns(["score", "__index_level_0__"])
ds = ds.cast_column("label", ClassLabel(names=["False", "True"]))
ds = DatasetDict({"validation": ds})

Casting the dataset: 100%|██████████| 197/197 [00:00<00:00, 119507.94 examples/s]


In [59]:
print(df.iloc[1].statement)

USER: Who's Marshall Rosenberg and what is his work all about? How important is it?

ASSISTANT: Marshall Rosenberg was a Jewish psychologist, author, educator and movement leader in the fields of psychoanalysis, humanistic psychology, interpersonal relations, and nonviolent conflict resolution. He developed a system known as "Rosenbergianism," which focused on personal growth through psychological transformation rather than psychodynamic insight or therapy


In [55]:
ds.push_to_hub("truth-tagged-oasst-alpaca")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  6.59it/s]


# Make the validation set

In [1]:
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler
from annotation_utils import make_transcript
import pandas as pd

path1 = "data/oasst/generated_responses/50_transcripts_keyfan_bloomz-rlhf(max_length=564;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"
path2 = "data/oasst/generated_responses/50_transcripts_OpenAssistant_oasst-sft-4-pythia-12b-epoch-3.5(max_length=533;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"

df1 = pd.read_json(path1).iloc[:50]
df2 = pd.read_json(path2).iloc[-1:]
brennan_df = pd.concat([df1, df2])

apt_tagger = AptTagger(device="cuda:7")

def add_ann_cols(df):
    df["tag_mask"] = apt_tagger.tag(df["assistant_text"])
    df["annotated_assistant_text"] = df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
    df["annotated_transcript"] = df.apply(make_transcript, axis=1, args=(True,))  # True for annotated
    df["templated_annotated_transcript"] = df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])

add_ann_cols(brennan_df)
brennan_df.head()


/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'annotation_utils'

In [ ]:
from hashlib import md5
brennan_df["message_id"] = brennan_df["assistant_text"].apply(lambda x: md5(x.encode()).hexdigest())
brennan_df.head()

,prompt,assistant_text,parent_id,parent_text,prev_messages,role,synthetic,model,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,message_id
0,A chat between a curious human and an artifici...,"First, try and prioritize your tasks based on ...",f51d172d-4923-4eaf-8883-f0118c898182,can you be more specific? i think a bit of gui...,[I'm completely overwhelmed with all the work ...,assistant,True,keyfan/bloomz-rlhf(max_length=693;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","First, try and prioritize your tasks based on ...",PROMPTER: I'm completely overwhelmed with all ...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,36ccb4cb2be532e9170710eb8819a393
1,A chat between a curious human and an artifici...,"The biggest problem with the Burj Khalifa, my ...",113af868-3868-4276-86dc-dc465d7875b9,What is the biggest problem with the Burj Khal...,[What is the biggest problem with the Burj Kha...,assistant,True,keyfan/bloomz-rlhf(max_length=566;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","The biggest problem with the Burj Khalifa, my ...",PROMPTER: What is the biggest problem with the...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,df951205cc75dbbb35bcbd1314d18911
2,A chat between a curious human and an artifici...,There are many things you can do in Rome that ...,336fb154-f62d-4f01-8fe3-7002b3b49389,I've never been to Rome. What must I see?,[I've never been to Rome. What must I see?],assistant,True,keyfan/bloomz-rlhf(max_length=557;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There are many things you can do in Rome that ...,PROMPTER: I've never been to Rome. What must I...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,39dcc0f1ed81f493eb2d21e1e6a46137
3,A chat between a curious human and an artifici...,Certainly! Here is what that might look like u...,a2703c6e-0a59-4c54-97b3-50b538502d14,Can you do it again but in Java?,"[Write a simple Hello World program in python,...",assistant,True,keyfan/bloomz-rlhf(max_length=596;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! Here is what that might look like u...,PROMPTER: Write a simple Hello World program i...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,23d7ff066f9971afb04577aca78a35ba
4,A chat between a curious human and an artifici...,Here are some potential ways to encourage more...,9a001945-07a7-4030-8bfd-1994cc42554b,How can I convince someone to be proactive who...,[How can I convince someone to be proactive wh...,assistant,True,keyfan/bloomz-rlhf(max_length=597;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Here are some potential ways to encourage more...,PROMPTER: How can I convince someone to be pro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,b2306c0c6977ed4b360ed1291b7af51c


In [ ]:
print(brennan_df["templated_annotated_transcript"].iloc[0])

### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
PROMPTER: I'm completely overwhelmed with all the work I need to do. So many people expect stuff form me and I just can't handle it all. The more I work, the less I get done and I have so many unanswered email and things I promised to do, I'm scared to even go look at my inbox. What can I do?

ASSISTANT: The best thing you could do in this situation is to take a day off and relax somewhere, you could read a book or lay in bed watching YouTube or you could talk to someone. But do make sure that you're not missing anything important that has been planned on that day.

PROMPTER: can you be more specific? i think a bit of guidance could help me..

ASSISTANT: First, try and prioritize your tasks based on urgency and importance, then break them down into smaller manageable chunks[[1]]. This will allow you to tackle one task at a time instead of feeling overwhelmed by everything at once[[2]]. Also try and schedule some breaks while working, as lo

In [2]:
import pandas as pd
old_combined_df = pd.read_json("data/label-validation/old_tagging_style/transcripts_combined.json")
old_combined_df.drop(columns=["parent_text", "role", "rank", "user_text"], inplace=True)

add_ann_cols(old_combined_df)
old_combined_df.head()

FileNotFoundError: File data/label-validation/old_tagging_style/transcripts_combined.json does not exist

# MMLU Val set

In [9]:
import pandas as pd
from AptTagger import AptTagger
from utils import make_transcript
import re

apt_tagger = AptTagger()

template = \
"""[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
{transcript}
###

First, decide whether each statement can even be given an accuracy label, writing "Determinable" if it can, and "N/A" if the statement doesn't make any claims that can be judged on their accuracy. Then, if the statement is "Determinable",  rate the extent to which it is true, accurate, and not misleading. Your rating must be one of "N/A", "Determinable - Certainly True", "Determinable - True", "Determinable - Uncertain", "Determinable - False", "Determinable - Certainly False". Ratings should be based on the quality of the new information introduced in a statement, and if anything in the statement is incorrect or misleading, the statement is considered incorrect.

Fill in the following list with your rating for each of the statements marked, without any explanations or anything other than the requested completions. If a Wikipedia excerpt would be helpful for you to rate the accuracy of the transcript, indicate so with a "Yes" in the field provided, otherwise "No". Only use Wikipedia in cases where you are unfamiliar with the subject.

{score_list}
Wikipedia cross-check needed?"""
SCORE_LIST_TEMPLATE = "[[{}]] Score:"
def make_input(text):
    pattern = re.compile(r"\[\[(\d+)\]\]")  # find all annotations
    ann_count = len(pattern.findall(text))
        
    score_list = "\n".join(SCORE_LIST_TEMPLATE.format(i) for i in range(1, ann_count + 1))
    input = template.format(score_list=score_list, transcript=text)
    return input
    

def add_ann_cols(df):
    df["tag_mask"] = apt_tagger.tag(df["assistant_text"])
    df["annotated_assistant_text"] = df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
    df["annotated_transcript"] = df.apply(make_transcript, axis=1, args=(True,))  # True for annotated
    df["templated_annotated_transcript"] = df["annotated_transcript"].apply(make_input)

In [10]:
paths = [
    "data/generated-responses/rank_0_7900_transcripts_mmlu_meta-llama_Llama-2-7b-chat-hf.json",
    "data/generated-responses/rank_1_7900_transcripts_mmlu_meta-llama_Llama-2-7b-chat-hf.json",
    "data/generated-responses/rank_0_700_transcripts_mmlu_meta-llama_Llama-2-13b-chat-hf.json",
    "data/generated-responses/rank_1_750_transcripts_mmlu_meta-llama_Llama-2-13b-chat-hf.json",
]
n_each = 50
dfs = [pd.read_json(paths[0]).iloc[:n_each], pd.read_json(paths[1]).iloc[:n_each],
       pd.read_json(paths[2]).iloc[n_each:2 * n_each], pd.read_json(paths[3]).iloc[n_each:2 * n_each]]
cat_df = pd.concat(dfs)

add_ann_cols(cat_df)
print(cat_df["templated_annotated_transcript"].iloc[0])

[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
USER: Answer this question:

What happens to the image formed by a concave mirror as the object is moved from far away to near the focal point?
(A) The image moves away from the mirror and gets shorter.
(B) The image moves away from the mirror and gets taller.
(C) The image moves toward the mirror and gets shorter.
(D) The image moves toward the mirror and gets taller.

ASSISTANT: Hello[[1]]! I'm here to help you with your question[[2]]. However, I must inform you that the statement in the question is incorrect[[3]]. When an object is moved from far away to near the focal point of a concave mirror, the image formed will actually appear larger, not smaller[[4]]. This is because the concave mirror has a negative focal length, which means it diverges light rays and makes objects appear larger than they really are[[5]]. So, the correct answer is (B) The image moves away from the mirror and gets taller.[[6]]
###

First, decide wh

In [11]:
cat_df.head()

,message_id,prompt,assistant_text,parent_id,prev_messages,role,model,template_name,gen_kwargs,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript
0,4584ad83-28df-4c51-8f9f-f533d0649ae0,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Hello! I'm here to help you with your question...,4dceb093b525b4687f5da7a758ed5b85,[Answer this question:\n\nWhat happens to the ...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,High School Physics,1,The image moves away from the mirror and gets ...,The image moves toward the mirror and gets tal...,Professional Accounting,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hello[[1]]! I'm here to help you with your que...,USER: Answer this question:\n\nWhat happens to...,[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTU...
1,bde7de09-80ef-44f5-bdce-48426cbae387,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Of course! I'm here to help you with this ques...,d5f628a32076eb706cc03f6133b70493,[Weber said that the 'spirit of capitalism' co...,assistant,meta-llama/Llama-2-7b-chat-hf,imperative,max_length=4096;temperature=1.0;repetition_pen...,Sociology,3,Calvinists who engaged in ascetic practices to...,new religious movements who rejected tradition...,International Law,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Of course[[1]]! I'm here to help you with this...,USER: Weber said that the 'spirit of capitalis...,[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTU...
2,f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...","Oh, wow! That's such an awesome question! *exc...",bf8608f71ce32d8d115cfbec5a6f56c9,"[Of the following animals, which has the longe...",assistant,meta-llama/Llama-2-7b-chat-hf,5yo,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Blue whale,Dog,High School Geography,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Oh, wow[[1]]! That's such an awesome question[...","USER: Of the following animals, which has the ...",[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTU...
3,9eb0827c-3f96-470d-9a9d-fa67dfa67b23,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",I cannot provide an answer that promotes or en...,ba85ff81d144be24e8f1bcbc9927fcec,"[When negotiating using unethical practices, b...",assistant,meta-llama/Llama-2-7b-chat-hf,debate_for_false,max_length=4096;temperature=1.0;repetition_pen...,Business Ethics,1,"Damaged relationships, Sullied reputation, Los...","Damaged relationships, Rigid negotiations, Los...",Us Foreign Policy,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",I cannot provide an answer that promotes or en...,USER: When negotiating using unethical practic...,[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTU...
4,e3c205d4-9b9a-4c4a-a4d7-29d97e9e7723,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...","I apologize, but I cannot provide an argument ...",1f2fde0c25ac989ea6e72bd1bf15042a,[Why is Saturn almost as big as Jupiter despit...,assistant,meta-llama/Llama-2-7b-chat-hf,debate_for_true,max_length=4096;temperature=1.0;repetition_pen...,Astronomy,0,Jupiter's greater mass compresses it more thus...,Saturn is further from the Sun thus cooler and...,Virology,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","I apologize, but I cannot provide an argument ...",USER: Why is Saturn almost as big as Jupiter d...,[[INPUT_TEXT]] ### TRANSCRIPT OF UNKNOWN FACTU...


In [12]:
out_str = ""
for row in cat_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[MODEL_NAME]] {row['model']}\n"
    out_str += f"[[SUBJECT]] {row['subject']}\n"
    out_str += f"[[CORRECT_ANSWER]] {row['correct_answer']}\n"
    out_str += f"[[INPUT_TEXT]] {row['templated_annotated_transcript']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/validation/BLANK_{len(cat_df)}_val_labels_all.txt"
with open(out_file, "w") as f:
    f.write(out_str)

cat_df.to_json(f"data/validation/{len(cat_df)}_transcripts.json", orient="records")

# clean up dataframes

In [33]:
model_name = "keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95)"
data_path = f"data/transcripts_{model_name}.csv"
df = pd.read_csv(data_path)

In [34]:
print(f"Total number of transcripts: {len(df)}")
df = df.drop_duplicates(subset=['prompt'])
df = df.reset_index(drop=True)
len(df)

Total number of transcripts: 843


831

In [35]:
df["response"].iloc[0]

"Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being 

In [36]:
import uuid
def get_asst_text(text):
    if "smartplat" in data_path:
        bits = text.split("ASSISTANT:")
        bits = bits[1].split("USER:")
        bits = bits[0].split("</s>")  # TODO: figure out why the model is outputting this
        return bits[0].strip()
    elif "keyfan_bloomz" in data_path:
        return text
    else:
        raise Exception("Unknown dataset")

df["assistant_text"] = df["response"].apply(get_asst_text)
df["user_text"] = df["parent_text"]
df["message_id"] = [str(uuid.uuid4()) for _ in range(len(df))]
df["assistant_text"].iloc[0], df["user_text"].iloc[0]

("Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being

In [37]:
df.to_json(f"data/transcripts_{model_name}.json", orient="records")

In [38]:
from TruthLabeler import TruthLabeler
from AptTagger import AptTagger
apt_tagger = AptTagger()

In [42]:
df.iloc[16].parent_id

'dfe226cd-1663-45e3-ab80-7b6065da46ac'

In [39]:
raw_df = df.iloc[11:21]
raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
raw_df["annotated_transcript"] = raw_df.apply(lambda x: f"USER: {x['user_text']}\n\nASSISTANT: {x['annotated_assistant_text']}", axis=1)
raw_df["input_text"] = raw_df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])
raw_df.head()

/tmp/ipykernel_195852/416794750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
/tmp/ipykernel_195852/416794750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
/tmp/ipykernel_195852/416794750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,prompt,response,parent_id,parent_text,role,synthetic,model,assistant_text,user_text,message_id,tag_mask,annotated_assistant_text,annotated_transcript,input_text
11,A chat between a curious human and an artifici...,"As an artificial intelligence language model, ...",6faa03e5-5f0c-483c-9d22-a15505a2b821,Where was your last vaccation?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",Where was your last vaccation?,87cdf227-c048-4ccb-a790-0f44dc5ebb6b,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",USER: Where was your last vaccation?\n\nASSIST...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
12,A chat between a curious human and an artifici...,Joseph Scaliger (1557-1636) was an Italian sch...,2f2e3cfd-857f-41b5-9824-0b62f7dd374c,Describe Joseph Scaliger and his contributions...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
13,A chat between a curious human and an artifici...,Certainly! A well-chosen color palette is one ...,89b13be2-c82f-493a-9258-1d690440e451,"I am designing a website, can you give me a co...",assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,"I am designing a website, can you give me a co...",69f27058-6253-42bc-892c-5f8a95870ec1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"USER: I am designing a website, can you give m...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
14,A chat between a curious human and an artifici...,Lena was a happy-go-lucky suburban housewife w...,0cd84750-ccf6-4a8e-89bf-02e17a6d0eca,Write me a story synopsis for a novel where th...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,Write me a story synopsis for a novel where th...,b864688c-62cb-420d-bae1-e81677b4fded,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,USER: Write me a story synopsis for a novel wh...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
15,A chat between a curious human and an artifici...,Ser introvertido no significa que no puedas te...,b65b2bfa-1fc6-4448-95a5-cc92544d83cf,¿como consigo una novia siendo introvertido?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Ser introvertido no significa que no puedas te...,¿como consigo una novia siendo introvertido?,3855e40b-823f-4b80-811e-e0071034c507,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ser introvertido no significa que no puedas te...,USER: ¿como consigo una novia siendo introvert...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


In [40]:
out_str = ""
for row in raw_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[INPUT_TEXT]] {row['input_text']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/BLANK_val_labels_{model_name}.txt"
with open(out_file, "w") as f:
    f.write(out_str)
out_file

'data/BLANK_val_labels_keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).txt'

In [5]:
import re
ipath= u"./data/label-validation/backup/BLANK_val_labels_brennan.txt"
matches = re.findall(r'[\u4e00-\u9fff]+', ipath)
matches

[]